### The Silence of the Integers

The following notebook highlights the problem of silent integer overrun in numpy and scipy. Both, numpy and scipy don't check integer overflow in arrays or matrices which has the consequence that one might end up with inconsistent data despite correct code.

In [ ]:
from scipy import sparse
from scipy import special
import numpy as np
import scipy

This code has been tested with the following versions of numpy and scipy:
    - scipy version=1.2.0
    - numpy version=1.16.0

In [ ]:
print(f'scipy version={scipy.__version__}')
print(f'numpy version={np.__version__}')

##### The scipy problem

Suppose we want to create a co-occurrence matrix from some observations. We've collected the base set of items (i.e. our vocabulary), so we can just record the row and column indices of the observed item - alongside its actual value.

As the code below shows, we're trying to be efficient about the size of the `dtype` we're using. Maybe through domain knowledge or other, we know that we're never going to encounter a value larger than 255, or maybe we cap any value at 100.

This is somewhat artificial, but highlights the actual problem. Theoretically, even a `np.uint64` might be too small for the number of observations you have.

The problem comes at element 2/2 - we have 2 observations that sum up to 256, but our dtype's max value is 255. Lets see what will happen.

In [ ]:
# Lets create some co-occurrence data
rows = np.array([1, 1, 1, 1, 2, 2, 3, 1, 1, 2], dtype=np.uint8)
cols = np.array([0, 1, 2, 3, 0, 2, 1, 0, 1, 2], dtype=np.uint8)
data = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 255], dtype=np.uint8)

In [ ]:
# And a sparse matrix for these observations
S = sparse.csr_matrix((data, (rows, cols)))
S.A

Well, _nothing_ happened. And that is precisely the problem. The element at 2/2 has silently overrun to 0.

But what if we explicitely specify a larger `dtype` when we create the sparse matrix? Surely this is going to prevent the overflow! After all, all values in our data array are `<= 255`! 

In [ ]:
S = sparse.csr_matrix((data, (rows, cols)), dtype=np.uint16)
S.A

Uh-oh...same problem again...not cool!

What we need to do is explicitly upcasting the data array, so any sort of gamble on using the smallest possible `dtype` for our data is more or less for nought.

In [ ]:
S = sparse.csr_matrix((data.astype(np.uint16), (rows, cols)))
S.A

##### More of the same

Suppose we're gathering our co-occurrence data periodically from some incoming stream and accumulate the data in "master" matrix

In [ ]:
# Lets create some more co-occurrence data, from some sort of incoming stream
rows = np.array([1, 1, 1, 1, 2, 2, 3, 1, 1, 2], dtype=np.uint8)
cols = np.array([0, 1, 2, 3, 0, 2, 1, 0, 1, 2], dtype=np.uint8)
data = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 13], dtype=np.uint8)

# S is the kind of master matrix, accumulating everything
S = sparse.csr_matrix((data, (rows, cols)))
S.A

So far, all looks well, lets wait for the next bunch of data from the stream...

In [ ]:
# Lets have some more co-occurrence data
rows = np.array([1, 2, 2, 2], dtype=np.uint8)
cols = np.array([0, 0, 2, 2], dtype=np.uint8)
data = np.array([1, 1, 1, 250], dtype=np.uint8)

T = sparse.csr_matrix((data, (rows, cols)), shape=S.shape)
T.A

The data look good, so just add them to our master matrix

In [ ]:
# Lets accumulate the co-occurrence data
U = S + T
U.A

Damn...position 2/2 again overflowed silently.

OK, you might argue, adding sparse matrices is really a bit of an edge case maybe. But what if we wanted to scale our data a bit? Lets recall the contents of `S` first, and then do the scaling.

In [ ]:
S.A

In [ ]:
V = S * 20
V.A

Damn, `14 * 20` is certainly _NOT_ `24`...

##### The numpy problem

But you know what, our data is not that sparse and not that big really. We can just use numpy instead of scipy. Surely, numpy is a bit more forgiving!

In [ ]:
Ud = S.toarray() + T.toarray()
Ud

What the actual F...???!!! But, but, lets try again...

In [ ]:
Vd = S.toarray() * 20
Vd

This can not be real??!! numpy just overflows as silently as scipy!! 

This must be the point where we declare that god is dead and become nihilists...

For numpy too, we need to explicitly upcast the dtype.

In [ ]:
Ud = S.toarray().astype(np.uint16) + T.toarray().astype(np.uint16)
Ud

In [ ]:
Vd = S.toarray().astype(np.uint16) * 20
Vd

Right, maybe we're just not very smart. So lets squiggle through the docs and see what we find. Ah yes, there it is, [numpy.seterr](https://docs.scipy.org/doc/numpy/reference/generated/numpy.seterr.html) looks precisely like the thing we're looking for! _Slightly_ inconvenient that the default behaviour is a silent overflow, but no worries, there is hope! As expected, the scalar silently overflows:

In [ ]:
np.uint8(255) * np.uint8(3)

But what if we just set the error to `raise`, this way we'll hit a hard error if any integer should overflow! Much better than all this silence!

In [ ]:
_ = np.seterr(over='raise')
np.uint8(255) * np.uint8(3)

Mmmmhhh, the beautiful smell of a fresh overflow error :D, looks like we're making progress! Lets go to our arrays again!

In [ ]:
Ud = S.toarray() + T.toarray()
Ud

**Wait, WHAT???!!!!** You mean, this doesn't work for arrays???!!!

In [ ]:
Vd = S.toarray() * 20
Vd

Nope, it doesn't! Lets try with scipy again and hope against hope that maybe scipy behaves differently!

In [ ]:
# scipy also offers some error behaviour customisation...
_ = special.seterr(overflow='raise')

In [ ]:
V = S * 20
V.A

No, no, no, silent integer overflows all over the place. Again, while the examples in this notebook are slightly artificial due to using `np.uint8`'s, they illustrate the problem. If you're using 32bit types, then overflows can happen quicker than you might think, `np.int32`'s max value is "only" `2147483647` and `np.uint32`'s max value is twice that (`4294967295`), so depending on how many observations you might have, this can be enough - or not. And when its not, it will become _really_ hard to find the error in your data. Oh, and obviously even the whooping max value of `np.uint64` (`18446744073709551615`) might silently overflow at some point...